<a href="https://colab.research.google.com/github/Sep-eg/kagglestudy/blob/main/titanic/BCT_lv1_2nd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🚢 EDA To Prediction(DieTanic)
이 노트북은 {AIB Alumni Challenge - Season2: 캐글 필사 스터디}를 위해 작성하였습니다.

코드의 출처는 [Kaggle, EDA To Prediction(DieTanic)](https://www.kaggle.com/ash316/eda-to-prediction-dietanic)으로 클릭하면 이동할 수 있습니다.

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle competitions download -c titanic

## Part1: Exploratory Data Analysis(EDA)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
data = pd.read_csv('train.csv')

In [ ]:
print('총 관측치 수:', len(data))

In [ ]:
data.head()

In [ ]:
#결측치 확인
data.isnull().sum()

Age, Cabin과 Embarked에 결측치가 있는것을 확인할 수 있습니다. 이후 처리하도록 하겠습니다.

In [ ]:
# 타겟인 Survived의 분포를 확인하기위해 시각화 합니다.
f, ax = plt.subplots(1, 2, figsize=(18,8))
data['Survived'].value_counts().plot.pie(explode=[0,0.1],
                                         autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Survived')
ax[0].set_ylabel('')
sns.countplot('Survived',data=data,ax=ax[1])
ax[1].set_title('Survived')
plt.show()

훈련 데이터셋의 891명의 탑승객중 38.4%, 약 350명 가량만이 살아남은 것으로 나타났습니다.

###1.1 Type Of Features
**Categorical Features**

범주형 변수는 둘 이상으로 분류할 범주가 있는 변수입니다. 예를 들어 성별은 남성과 여성, 두 가지로 분류할 수 있는 범주형 변수입니다.

특히 ***순서를 정할 수 없는 분류로 나누어진 변수***를 말합니다.

**해당 데이터셋의 범주형 특성: Sex, Embarked**

**Ordinal Features**

순서형 변수는 범주형과 유사하지만, ***범주간의 상대적인 순서가 존재하여 정렬이 가능한 변수***입니다. 예를 들어, 높이를 높다와 낮다 두 가지의 분류로 나눈다면, 이는 정렬할 수 있는 순서형 변수입니다.

**해당 데이터셋의 순서형 특성: Pclass**

**Continous Features**

연속형 변수는 아무런 두 점 사이 혹은 최솟값과 최댓값의 사이에서 취할 수 있는 값을 가진 변수를 말합니다.

**해당 데이터셋의 연속형 특성: Age**


###1.2 Analysing The Features

####1.2.1 Sex-> Categorical Feature

In [ ]:
data.groupby(['Sex', 'Survived'])['Survived'].count()

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18,8))
data[['Sex', 'Survived']].groupby(['Sex']).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived vs Sex')
sns.countplot('Sex', hue='Survived', data=data, ax=ax[1])
ax[1].set_title('Sex:Survived vs Dead')
plt.show()

전체 배의 탑승자는 남성이 여성에 비해 훨씬 많습니다. 하지만 생존자의 경우 여성의 비율이 훨씬 높으며, 여성의 생존률은 약 75%에 달하지만, 남성의 생존률은 약 18~19%입니다.

모델링에 매우 중요한 특성일 것으로 생각되며 다른 특성도 살펴보겠습니다.

####1.2.2 Pclass->Ordinal Feature

In [ ]:
pd.crosstab(data.Pclass,data.Survived,margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
f, ax = plt.subplots(1,2, figsize=(18,8))
data['Pclass'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'], ax=ax[0])
ax[0].set_title('Number Of Passengers By Pclass')
ax[0].set_ylabel('Count')
sns.countplot('Pclass', hue='Survived', data=data, ax=ax[1])
ax[1].set_title('Pclass:Survived vs Dead')
plt.show()

탑승권 클래스가 높을수록 생존률이 높은 것을 확인할 수 있습니다.

In [ ]:
pd.crosstab([data.Sex, data.Survived], data.Pclass, margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
sns.factorplot('Pclass', 'Survived', hue='Sex', data=data)

모든 클래스에서 여성의 생존률이 남성에 비해 높은것으로 나타났습니다.

특히 여성이면서 Pclass가 1인 경우 생존률이 약 95~96%로 매우 높았습니다.

####1.2.3 Age->Continous Feature

In [ ]:
print('Oldest Passenger was of:',data['Age'].max(),'Years')
print('Youngest Passenger was of:',data['Age'].min(),'Years')
print('Average Age on the ship:',data['Age'].mean(),'Years')

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
sns.violinplot("Pclass","Age", hue="Survived", data=data,split=True,ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0,110,10))
sns.violinplot("Sex","Age", hue="Survived", data=data,split=True,ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0,110,10))
plt.show()